In [1]:
import pandas as pd
import pickle
import os
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
# Шаг 1: Загрузка данных
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

In [3]:
# Загрузка данных из таблицы user_data
user_data_query = "SELECT * FROM public.user_data"
user_data = pd.read_sql(user_data_query, engine)

In [8]:
user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [5]:
# Загрузка данных из таблицы post_text_df
post_text_query = "SELECT * FROM public.post_text_df"
post_text_df = pd.read_sql(post_text_query, engine)

In [6]:
post_text_df

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [ ]:
feed_data_query = "SELECT * FROM public.feed_data"
feed_data = pd.read_sql(feed_data_query, engine)

In [8]:
feed_data

,timestamp,user_id,post_id,action,target
0,2021-10-07 18:13:00,166773,5008,view,1
1,2021-10-07 18:13:38,166773,5008,like,0
2,2021-10-07 18:13:40,166773,3679,view,0
3,2021-10-07 18:15:38,166773,4010,view,0
4,2021-10-07 18:18:03,166773,1777,view,0
...,...,...,...,...,...
995,2021-10-15 13:06:51,166773,373,view,0
996,2021-10-15 13:08:01,166773,5239,view,0
997,2021-10-15 13:09:40,166773,1819,view,0
998,2021-10-15 13:12:21,166773,2853,view,0


In [5]:
# Загрузка данных из таблицы user_data
user_data_query = "SELECT * FROM public.user_data"
user_data = pd.read_sql(user_data_query, engine)

# Загрузка данных из таблицы post_text_df
post_text_query = "SELECT * FROM public.post_text_df"
post_text_df = pd.read_sql(post_text_query, engine)

# Загрузка части данных из таблицы feed_data
feed_data_query = "SELECT * FROM public.feed_data LIMIT 1000"
feed_data = pd.read_sql(feed_data_query, engine)

In [16]:
user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [17]:
post_text_df.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [18]:
feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-12-02 18:50:32,116900,1454,view,0
1,2021-12-02 18:52:14,116900,1280,view,0
2,2021-12-03 09:00:16,116900,2946,view,0
3,2021-12-03 09:01:47,116900,3458,view,0
4,2021-12-03 09:04:22,116900,7131,view,0


In [9]:
merged_data = feed_data.merge(user_data, on='user_id', how='left')

In [ ]:
merged_data = merged_data.merge(post_text_df, left_on='post_id', right_on='post_id', how='left')

In [20]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  1000 non-null   datetime64[ns]
 1   user_id    1000 non-null   int64         
 2   post_id    1000 non-null   int64         
 3   action     1000 non-null   object        
 4   target     1000 non-null   int64         
 5   gender     1000 non-null   int64         
 6   age        1000 non-null   int64         
 7   country    1000 non-null   object        
 8   city       1000 non-null   object        
 9   exp_group  1000 non-null   int64         
 10  os         1000 non-null   object        
 11  source     1000 non-null   object        
 12  text       1000 non-null   object        
 13  topic      1000 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(7)
memory usage: 117.2+ KB


In [43]:
categorical_columns = merged_data.loc[:,merged_data.dtypes==object].columns

In [23]:
numeric_columns = merged_data.loc[:,merged_data.dtypes!=object].columns

In [29]:
numeric_columns = numeric_columns.drop('target')

In [50]:
import pandas as pd

# Assuming merged_data is your DataFrame and numeric_columns includes 'timestamp'
# Convert the 'timestamp' column to datetime format
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'])

# Sort the DataFrame by the 'timestamp' column
merged_data = merged_data.sort_values(by='timestamp')

# Extract various components from the 'timestamp' column
merged_data['year'] = merged_data['timestamp'].dt.year
merged_data['month'] = merged_data['timestamp'].dt.month
merged_data['day'] = merged_data['timestamp'].dt.day
merged_data['hour'] = merged_data['timestamp'].dt.hour
merged_data['minute'] = merged_data['timestamp'].dt.minute
merged_data['second'] = merged_data['timestamp'].dt.second

              timestamp  user_id  post_id action  target  gender  age country  \
759 2021-10-01 16:19:03    88874     5392   view       1       1   20  Russia   
760 2021-10-01 16:19:44    88874     5392   like       0       1   20  Russia   
761 2021-10-01 16:19:46    88874     1768   view       0       1   20  Russia   
762 2021-10-01 16:20:15    88874      842   view       0       1   20  Russia   
763 2021-10-01 16:22:25    88874     1818   view       0       1   20  Russia   

       city  exp_group   os source  \
759  Shatsk          3  iOS    ads   
760  Shatsk          3  iOS    ads   
761  Shatsk          3  iOS    ads   
762  Shatsk          3  iOS    ads   
763  Shatsk          3  iOS    ads   

                                                  text          topic  year  \
759  If this was the best dutch cinema had to offer...          movie  2021   
760  If this was the best dutch cinema had to offer...          movie  2021   
761  Holmes back on form in Birmingham\n\nDoubl

In [53]:
merged_data

,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic,year,month,day,hour,minute,second
759,2021-10-01 16:19:03,88874,5392,view,1,1,20,Russia,Shatsk,3,iOS,ads,If this was the best dutch cinema had to offer...,movie,2021,10,1,16,19,3
760,2021-10-01 16:19:44,88874,5392,like,0,1,20,Russia,Shatsk,3,iOS,ads,If this was the best dutch cinema had to offer...,movie,2021,10,1,16,19,44
761,2021-10-01 16:19:46,88874,1768,view,0,1,20,Russia,Shatsk,3,iOS,ads,Holmes back on form in Birmingham\n\nDouble Ol...,sport,2021,10,1,16,19,46
762,2021-10-01 16:20:15,88874,842,view,0,1,20,Russia,Shatsk,3,iOS,ads,Media seek Jackson juror notes\n\nReporters co...,entertainment,2021,10,1,16,20,15
763,2021-10-01 16:22:25,88874,1818,view,0,1,20,Russia,Shatsk,3,iOS,ads,Rochus shocks Coria in Auckland\n\nTop seed Gu...,sport,2021,10,1,16,22,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,2021-12-23 11:26:18,88872,1523,view,1,0,19,Russia,Saint Petersburg,2,iOS,ads,Hodges announces rugby retirement\n\nScarlets ...,sport,2021,12,23,11,26,18
600,2021-12-23 11:28:38,88872,1523,like,0,0,19,Russia,Saint Petersburg,2,iOS,ads,Hodges announces rugby retirement\n\nScarlets ...,sport,2021,12,23,11,28,38
601,2021-12-23 11:28:40,88872,1127,view,0,0,19,Russia,Saint Petersburg,2,iOS,ads,UK will stand firm on EU rebate\n\nBritains £3...,politics,2021,12,23,11,28,40
602,2021-12-23 11:31:37,88872,435,view,0,0,19,Russia,Saint Petersburg,2,iOS,ads,Ukraine revisits state sell-offs\n\nUkraine is...,business,2021,12,23,11,31,37


In [55]:
merged_data = merged_data.drop(['timestamp', 'user_id','post_id'], axis=1)

In [57]:
X = merged_data.drop('target',axis=1)
y = merged_data['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
unique_values = merged_data[categorical_columns].nunique()
one_hot_columns = unique_values[unique_values < 5].index.tolist()
target_encoder_columns = unique_values[(unique_values >= 5) & (unique_values.index != 'text')].index.tolist()
text_column = 'text' 

In [62]:
categorical_preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), one_hot_columns),
        ('target', TargetEncoder(), target_encoder_columns),
        ('tfidf', TfidfVectorizer(), text_column)
    ],
    remainder='passthrough'
)

In [63]:
from sklearn.pipeline import Pipeline

In [76]:
# Create a pipeline that first applies the categorical preprocessor and then the scaler
pipe = Pipeline(steps=[
    ('categorical', categorical_preprocessor),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', LogisticRegression())
])

In [77]:
pipe.fit(X_train, y_train)

/Users/ivangolov/Desktop/KC/Final project/Final_project_part2/KC_project_II/.venv/lib/python3.8/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/Users/ivangolov/Desktop/KC/Final project/Final_project_part2/KC_project_II/.venv/lib/python3.8/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/Users/ivangolov/Desktop/KC/Final project/Final_project_part2/KC_project_II/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.



Pipeline(steps=[('categorical',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['action', 'country', 'os',
                                                   'source']),
                                                 ('target', TargetEncoder(),
                                                  ['city', 'topic']),
                                                 ('tfidf', TfidfVectorizer(),
                                                  'text')])),
                ('scaler', StandardScaler(with_mean=False)),
                ('model', LogisticRegression())])

In [4]:
def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH

def save_model(model, path: str):
    model_path = get_model_path(path)
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)

def load_models():
    model_path = get_model_path("model.pkl")
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model


loaded_model = load_models()

In [5]:
loaded_model 

Pipeline(steps=[('categorical',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['action', 'country', 'os',
                                                   'source']),
                                                 ('target', TargetEncoder(),
                                                  ['city', 'topic']),
                                                 ('tfidf', TfidfVectorizer(),
                                                  'text')])),
                ('scaler', StandardScaler(with_mean=False)),
                ('model', LogisticRegression())])

In [6]:
assert hasattr(loaded_model, "predict")
assert hasattr(loaded_model, "predict_proba")

In [78]:
preds = pipe.predict(X_test)

In [79]:
from sklearn.metrics import classification_report

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.87      0.93      0.89       174
           1       0.07      0.04      0.05        26

    accuracy                           0.81       200
   macro avg       0.47      0.48      0.47       200
weighted avg       0.76      0.81      0.78       200



In [12]:
import pandas as pd
from sqlalchemy import create_engine

def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def save_features():
    # Создаем соединение с базой данных
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )

    # Загрузка данных из таблицы user_data
    user_data_query = "SELECT * FROM public.user_data"
    user_data = batch_load_sql(user_data_query)

    # Загрузка данных из таблицы post_text_df
    post_text_query = "SELECT * FROM public.post_text_df"
    post_text_df = batch_load_sql(post_text_query)

    # Загрузка части данных из таблицы feed_data
    feed_data_query = "SELECT * FROM public.feed_data LIMIT 1000"
    feed_data = batch_load_sql(feed_data_query)

    # Объединение данных
    merged_data = user_data.merge(feed_data, on='user_id', how='left')
    merged_data = merged_data.merge(post_text_df, on='post_id', how='left')

    # Запись объединенных данных в новую таблицу
    table_name = 'ivan_golov_features_lesson_22'
    merged_data.to_sql(table_name, con=engine, if_exists='replace', index=False)

def load_features() -> pd.DataFrame:
    query = "SELECT * FROM ivan_golov_features_lesson_22"
    return batch_load_sql(query)

In [13]:
save_features()
features = load_features()
print(features.head())

   user_id  gender  age country               city  exp_group       os source  \
0      200       1   34  Russia          Degtyarsk          3  Android    ads   
1      201       0   37  Russia             Abakan          0  Android    ads   
2      202       1   17  Russia           Smolensk          4  Android    ads   
3      203       0   18  Russia             Moscow          1      iOS    ads   
4      204       0   36  Russia  Anzhero-Sudzhensk          3  Android    ads   

  timestamp  post_id action  target  text topic  
0       NaT      NaN   None     NaN  None  None  
1       NaT      NaN   None     NaN  None  None  
2       NaT      NaN   None     NaN  None  None  
3       NaT      NaN   None     NaN  None  None  
4       NaT      NaN   None     NaN  None  None  


In [14]:
features

,user_id,gender,age,country,city,exp_group,os,source,timestamp,post_id,action,target,text,topic
0,200,1,34,Russia,Degtyarsk,3,Android,ads,NaT,NaN,None,NaN,None,None
1,201,0,37,Russia,Abakan,0,Android,ads,NaT,NaN,None,NaN,None,None
2,202,1,17,Russia,Smolensk,4,Android,ads,NaT,NaN,None,NaN,None,None
3,203,0,18,Russia,Moscow,1,iOS,ads,NaT,NaN,None,NaN,None,None
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,NaT,NaN,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164190,168548,0,36,Russia,Kaliningrad,4,Android,organic,NaT,NaN,None,NaN,None,None
164191,168549,0,18,Russia,Tula,2,Android,organic,NaT,NaN,None,NaN,None,None
164192,168550,1,41,Russia,Yekaterinburg,4,Android,organic,NaT,NaN,None,NaN,None,None
164193,168551,0,38,Russia,Moscow,3,iOS,organic,NaT,NaN,None,NaN,None,None
